In [1]:
"""
run 07_effective_trial_factor.py
"""

'\nrun 07_effective_trial_factor.py\n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [3]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> 06.txt
# 0: grb_name
# 1: n_inj
# 2: n_trials
# 3: tw_in_second
# 4: ncpu
# 5: batchIndex
# 6: use_poisson

# Check 1 out of 2

In [10]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/07.sub'
## [::-1] reverse order
grb_name_idx_starts = np.r_[0:2268:100]
grb_name_idx_ends   = np.r_[100:2268:100, 2268]
#######################################

In [11]:
contents = ""
for grb_idx_start, grb_idx_end in zip(grb_name_idx_starts, grb_name_idx_ends):   
    contents += "{}\t".format(grb_idx_start)
    contents += "{}\n".format(grb_idx_end)

with open("07.txt", 'w') as writer:
    writer.write(contents)
    

In [12]:
print(contents)

0	100
100	200
200	300
300	400
400	500
500	600
600	700
700	800
800	900
900	1000
1000	1100
1100	1200
1200	1300
1300	1400
1400	1500
1500	1600
1600	1700
1700	1800
1800	1900
1900	2000
2000	2100
2100	2200
2200	2268



In [13]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 23



In [14]:
### create pbs file ---> 07.pbs

# Check 2 out of 2

In [15]:
nameofjob  = "07"
ntotal     = 23           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "07.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 1  # in Hour

In [16]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
contents += "#PBS -l nodes=1:ppn={}\n".format(1)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/07.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "python 07_effective_trial_factor.py "

contents += "--grb_idx_start $arg0 "
contents += "--grb_idx_end $arg1 "
print(contents)
with open("07.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 07
#PBS -l nodes=1:ppn=1
#PBS -l pmem=4gb
#PBS -l walltime=1:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/07.out
#PBS -t 1-23%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/07.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
python 07_effective_trial_factor.py --grb_idx_start $arg0 --grb_idx_end $arg1 
